# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv("../data/customer_product_sales.csv")

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [15]:
quant_by_cust = data.groupby(["CustomerID", "ProductName"])["Quantity"].agg('sum')
quant_by_cust
customerstotal = pd.DataFrame(quant_by_cust)
customerstotal.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
table = pd.pivot_table(data, values='Quantity', index=['ProductName'], columns=['CustomerID'], aggfunc=np.sum, fill_value=0)
table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
Apricots - Halves,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,25,50,25,0,25,25
Apricots Fresh,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25
Arizona - Green Tea,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,25,0,0,0,0
Artichokes - Jerusalem,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
from scipy.spatial.distance import pdist, squareform

squareform(pdist(table.T, 'euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)

#distances
similarities = distances[200].sort_values(ascending=False)[1:].head() # Se selecciona desde 1 porque con el que más similarities tiene es él mismo.
similarities
#distances.head()
#distances[distances.CustomerID==200
#distances.iloc[1]

CustomerID
3317    0.084959
1072    0.083972
3535    0.082403
1920    0.082100
3909    0.080920
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [46]:
#data.loc[similarities.index]
step55 = quant_by_cust.loc[similarities.index]
step5 = pd.DataFrame(step55)
step5.head()

Quantity
CustomerID ProductName                                
1072       Appetizer - Mini Egg Roll, Shrimp         1
           Appetizer - Sausage Rolls                 1
           Beans - Kidney, Canned                    1
           Beef - Montreal Smoked Brisket            1
           Beef - Striploin Aa                       1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [45]:
step6 = step5.groupby(['ProductName']).sum()['Quantity'].sort_values(ascending = False)
productranking = pd.DataFrame(step6)
productranking.head()

,Quantity
ProductName,
Soup - Campbells Bean Medley,4
Muffin - Carrot Individual Wrap,3
Bay Leaf,3
Pork - Kidney,3
"Pepper - Black, Whole",3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [42]:
chosencustomer = customerstotal.loc[200]
chosencustomer.shape

(72, 1)

In [43]:
merge_recommended = pd.merge(chosencustomer, productranking, how='right', on='ProductName')
merge_recommended = merge_recommended.fillna(0)
merge_recommended.head()

,Quantity_x,Quantity_y
ProductName,,
Bacardi Breezer - Tropical,1.0,2
Beef - Ground Medium,1.0,1
Beef - Montreal Smoked Brisket,1.0,1
Beef - Texas Style Burger,1.0,1
Beef Ground Medium,1.0,1


In [44]:
recommended_product = (merge_recommended.loc[(merge_recommended['Quantity_x'] == 0)]).sort_values(by = ['Quantity_x'], ascending = False)
recommended_product.head() # Quantity_x es el sumatorio de Quantity purchased by chosen customer, customer 200. Lo ordenamos por 0, las que no ha comprado, de mínimo a max


,Quantity_x,Quantity_y
ProductName,,
Soup - Campbells Bean Medley,0.0,4
Skirt - 29 Foot,0.0,1
"Shrimp - Baby, Warm Water",0.0,1
Sea Bass - Whole,0.0,1
Sausage - Breakfast,0.0,1


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [91]:
dic = {}
lista = list(data.CustomerID)

for cust in lista:
    similarities = distances[cust].sort_values(ascending=False)[1:6] # Step 4
    similar_customers = customerstotal.loc[similarities.index] # Step 5
    ranked_products = similar_customers.groupby(["ProductName"]).sum().Quantity.sort_values(ascending=False)
    recommended_product = ranked_products[~ranked_products.index.isin(customerstotal.loc[cust].index)].head()
    dic[cust] = recommended_product.to_dict()
    
for cust in dic:
    dic[cust] = [x for x in dic[cust]]

In [93]:
'''
customers_list = list(set(data['CustomerID'])) #Creamos una lista de todos los Customers ID

def recommender(customer):
    
    similarities = distances[customer].sort_values(ascending=False)[1:6] # Step 4
    similar_customers = similarities.index[1:6] # Step 4
    
    def selecting(similar_customers): # Step 5
        selecting_customers = customerstotal.iloc[similar_customers]
        return selecting_customers
    
    ranked_products = (selecting(similar_customers).groupby(['ProductName'])['Quantity'].sum().sort_values(ascending = False)).to_frame() # Step 6. .to_frame() convierte series en DataFrame
    
    chosencustomer = customerstotal.loc[customer] # Step 7
    merge_recommended = (pd.merge(chosencustomer, ranked_products, how='right', on='ProductName')) #Step 7
    merge_recommended = merge_recommended.fillna(0)
    
    recommended_product = (merge_recommended[merge_recommended['Quantity_x'] == 0]).sort_values(by = ['Quantity_x'], ascending = False) # Step 7
    products = list(recommended_product.index)
    return products

def form_dict(customer):
    recommendation_dict[customer] = recommender(customer)
    return recommendation_dict

recommendation_dict = {}

for cust in customers_list:
    recommendation_dict.update(form_dict(cust))
    
recommendation_dict # ufff ha costado
'''

"\ncustomers_list = list(set(data['CustomerID'])) #Creamos una lista de todos los Customers ID\n\ndef recommender(customer):\n    \n    similarities = distances[customer].sort_values(ascending=False)[1:6] # Step 4\n    similar_customers = similarities.index[1:6] # Step 4\n    \n    def selecting(similar_customers): # Step 5\n        selecting_customers = customerstotal.iloc[similar_customers]\n        return selecting_customers\n    \n    ranked_products = (selecting(similar_customers).groupby(['ProductName'])['Quantity'].sum().sort_values(ascending = False)).to_frame() # Step 6. .to_frame() convierte series en DataFrame\n    \n    chosencustomer = customerstotal.loc[customer] # Step 7\n    merge_recommended = (pd.merge(chosencustomer, ranked_products, how='right', on='ProductName')) #Step 7\n    merge_recommended = merge_recommended.fillna(0)\n    \n    recommended_product = (merge_recommended[merge_recommended['Quantity_x'] == 0]).sort_values(by = ['Quantity_x'], ascending = False) #

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [95]:
final_reco = pd.DataFrame(dic).T
final_reco = final_reco.rename(index = str, columns = {0: "Producto 1", 1: "Producto 2", 2: "Producto 3", 3: "Producto 4", 4: "Producto 5"})
final_reco.head(10)


,Producto 1,Producto 2,Producto 3,Producto 4,Producto 5
61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,"Water - Mineral, Natural"
77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,"Oregano - Dry, Rubbed",Halibut - Steaks
23548,Squid - Tubes / Tenticles 10/20,Flavouring - Orange,Bread - Raisin Walnut Oval,Sprouts - Baby Pea Tendrils,Banana Turning
78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Beef - Inside Round
83106,Cocoa Butter,"Cheese - Boursin, Garlic / Herbs",Garlic - Peeled,Cheese - Mix,Sauce - Hollandaise
11253,Juice - Lime,Pomello,Tomatoes Tear Drop,"Potatoes - Instant, Mashed",Soup - Campbells Tomato Ravioli
35107,Bandage - Flexible Neon,Bread - French Baquette,Wine - Hardys Bankside Shiraz,Onion Powder,"Lentils - Red, Dry"
15088,Soupfoamcont12oz 112con,Soup Knorr Chili With Beans,Veal - Sweetbread,Wine - Fume Blanc Fetzer,Beef - Texas Style Burger
26031,Cheese - Mozzarella,"Lemonade - Natural, 591 Ml",Bananas,Quiche Assorted,Soup Knorr Chili With Beans


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [97]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'correlation'))), 
                         index=table.columns, columns=table.columns)

dic = {}
lista = list(data.CustomerID)

for cust in lista:
    similarities = distances[cust].sort_values(ascending=False)[1:6] # Step 4
    similar_customers = customerstotal.loc[similarities.index] # Step 5
    ranked_products = similar_customers.groupby(["ProductName"]).sum().Quantity.sort_values(ascending=False)
    recommended_product = ranked_products[~ranked_products.index.isin(customerstotal.loc[cust].index)].head()
    dic[cust] = recommended_product.to_dict()
    
for cust in dic:
    dic[cust] = [x for x in dic[cust]]

final_reco = pd.DataFrame(dic).T
final_reco = final_reco.rename(index = str, columns = {0: "Producto 1", 1: "Producto 2", 2: "Producto 3", 3: "Producto 4", 4: "Producto 5"})
final_reco.head(10)

,Producto 1,Producto 2,Producto 3,Producto 4,Producto 5
61288,Jagermeister,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
77352,Fondant - Icing,Grenadine,Mangoes,Flour - Whole Wheat,Lettuce - Frisee
40094,Truffle Cups - Brown,"Bread - Roll, Soft White Round",Juice - V8 Splash,"Veal - Brisket, Provimi,bnls",Anchovy Paste - 56 G Tube
23548,Olives - Stuffed,Wine - Charddonnay Errazuriz,Soup - Campbells Tomato Ravioli,Assorted Desserts,Juice - Orange
78981,"Tart Shells - Sweet, 4",Artichokes - Jerusalem,Sprouts - Baby Pea Tendrils,Chocolate - Dark,"Appetizer - Mini Egg Roll, Shrimp"
83106,Pastry - Cheese Baked Scones,Bouq All Italian - Primerba,Onions - Vidalia,Bread - Italian Roll With Herbs,"Turnip - White, Organic"
11253,Sauce - Demi Glace,"Nut - Chestnuts, Whole","Beef - Tenderlion, Center Cut","Mushroom - Trumpet, Dry","Soup - Canadian Pea, Dry Mix"
35107,Fond - Neutral,Snapple Lemon Tea,Truffle Cups - Brown,"Veal - Brisket, Provimi,bnls",Cheese Cloth No 100
15088,Browning Caramel Glace,Vaccum Bag 10x13,Pastry - Choclate Baked,Wine - Fume Blanc Fetzer,Skirt - 29 Foot
26031,Spinach - Baby,Foam Cup 6 Oz,General Purpose Trigger,Banana Turning,Smirnoff Green Apple Twist


In [98]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'consine'))), 
                         index=table.columns, columns=table.columns)

dic = {}
lista = list(data.CustomerID)

for cust in lista:
    similarities = distances[cust].sort_values(ascending=False)[1:6] # Step 4
    similar_customers = customerstotal.loc[similarities.index] # Step 5
    ranked_products = similar_customers.groupby(["ProductName"]).sum().Quantity.sort_values(ascending=False)
    recommended_product = ranked_products[~ranked_products.index.isin(customerstotal.loc[cust].index)].head()
    dic[cust] = recommended_product.to_dict()
    
for cust in dic:
    dic[cust] = [x for x in dic[cust]]

final_reco = pd.DataFrame(dic).T
final_reco = final_reco.rename(index = str, columns = {0: "Producto 1", 1: "Producto 2", 2: "Producto 3", 3: "Producto 4", 4: "Producto 5"})
final_reco.head(10)

ValueError: Unknown Distance Metric: consine